# permits-data

Data analysis pipeline for construction permits in Los Angeles.

For more information:
https://data.lacity.org/A-Prosperous-City/Building-and-Safety-Permit-Information/yv23-pmwf

In [239]:
import os
import sys
import pandas as pd
import psycopg2

In [240]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [241]:
sys.path[0] = '../'

## 1. Import Data

In [242]:
raw_data = 'permits_raw.csv'

URL = "https://data.lacity.org/api/views/yv23-pmwf/rows.csv?accessType=DOWNLOAD"
DATA_PATH = sys.path[0] + 'data/raw/' + raw_data

In [243]:
#data = pd.read_csv(DATA_PATH)

In [244]:
conn = psycopg2.connect(dbname='permits',
                       user='postgres',
                       password='password',
                        host='localhost', 
                        port=5432)

#sql = 'SELECT * FROM permits_raw WHERE "Zip Code" IN (90012, 90013, 90014, 90015, 90017, 90071) LIMIT 1000;'
sql_all = 'SELECT * FROM permits_raw;'

# Extract full dataset
#data = pd.read_sql_query(sql, conn)

### 1.1 Update Column Names in Postgres Database

In [245]:
# Get raw data column names
def get_table_names(table):
    sql = "SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'{}'".format(table)
    etl = pd.read_sql_query(sql, conn)
    old_columns = etl['column_name']
    
    return old_columns

In [246]:
# Retrieve table column names
old_columns = get_table_names("permits_raw")

In [247]:
# Rename columns, will update table later
def format_names(series):
    # Replace whitespace with underscore
    series = series.str.replace(' ', '_')

    # Replace hyphen with underscore
    series = series.str.replace('-', '_')

    # Replace hashtag with No (short for number)
    series = series.str.replace('#', 'No')

    # Replace forward slash with underscore
    series = series.str.replace('/', '_')

    # Remove period
    series = series.str.replace('.', '')

    # Remove open parenthesis
    series = series.str.replace('(', '')

    # Remove closed parenthesis
    series = series.str.replace(')', '')

    # Remove apostrophe
    series = series.str.replace("'", '')
    
    return series.str.lower()

In [248]:
# Transform table column names for permits_raw
new_columns = format_names(old_columns);

In [249]:
# Creates a SQL query to update table columns and writes to text file
def create_query(old_columns, new_columns, update_db=False):
    
    sql = 'ALTER TABLE permits_raw RENAME "{old_name}" to {new_name};'
    
    sql_query = []

    for idx, name in old_columns.iteritems():
        #print(idx, name)
        sql_query.append(sql.format(old_name=name, new_name=new_columns[idx]))
        
    update_names = '\n'.join(sql_query)
    # update later: sql_file = os.path.join(os.path.dirname(__file__), "../postgres/scripts/update_names.sql")
    with open('../postgres/scripts/update_names.sql', 'w') as text:
        text.write(update_names)
        
    if update_db==True:
        cur = conn.cursor()
        sql_file = open('../postgres/scripts/update_names.sql', 'r')
        cur.execute(sql_file.read())

In [250]:
# Create SQL query for permits_raw
create_query(old_columns, new_columns, update_db=True)

In [251]:
# Check table names are updated
get_table_names("permits_raw").head()

0      assessor_book
1      assessor_page
2    assessor_parcel
3              tract
4              block
Name: column_name, dtype: object

In [252]:
# Extract full dataset
data = pd.read_sql_query(sql_all, conn)

In [253]:
data.head()

,assessor_book,assessor_page,assessor_parcel,tract,block,lot,reference_no_old_permit_no,pcis_permit_no,status,status_date,permit_type,permit_sub_type,permit_category,project_number,event_code,initiating_office,issue_date,address_start,address_fraction_start,address_end,address_fraction_end,street_direction,street_name,street_suffix,suffix_direction,unit_range_start,unit_range_end,zip_code,work_description,valuation,floor_area_la_zoning_code_definition,no_of_residential_dwelling_units,no_of_accessory_dwelling_units,no_of_stories,contractors_business_name,contractor_address,contractor_city,contractor_state,license_type,license_no,principal_first_name,principal_middle_name,principal_last_name,license_expiration_date,applicant_first_name,applicant_last_name,applicant_business_name,applicant_address_1,applicant_address_2,applicant_address_3,zone,occupancy,floor_area_la_building_code_definition,census_tract,council_district,latitude_longitude,applicant_relationship,existing_code,proposed_code
0,4317,003,***,TR 30210-C,None,LT 1,None,15044-90000-08405,Permit Finaled,09/10/2015,HVAC,1 or 2 Family Dwelling,No Plan Check,None,None,INTERNET,2015-08-18,1823,1/2,1823,1/2,S,THAYER,AVE,None,None,None,90025.0,None,NaN,None,None,None,None,CONDITIONED AIRE MECHANICAL & ENGINEERING INC,18650 PARTHENIA STREET,NORTHRIDGE,CA,C20,532440,BRETT,MOORE,HOFFER,2016-06-30,BRETT,HOFFER,None,18650 PARTHENIA ST,None,"NORTHRIDGE, CA",R3-1-O,None,0,2671.00,5,"(34.05474, -118.42628)",Net Applicant,NaN,NaN
1,5005,010,017,CHESTERFIELD SQUARE,None,465,16SL57806,16016-70000-02464,Permit Finaled,08/01/2017,Bldg-Alter/Repair,1 or 2 Family Dwelling,No Plan Check,None,None,SOUTH LA,2016-02-04,2122,None,2122,None,W,54TH,ST,None,None,None,90062.0,General rehabilitation for single family dwell...,40000.0,None,None,None,None,OWNER-BUILDER,None,None,None,NA,0,JAVIER,None,TALAMANTES,None,JAVIER,TALAMANTES,OWNER-BUILDER,None,None,None,C2-1VL,None,None,2325.00,8,"(33.99307, -118.31668)",Owner-Bldr,1.0,NaN
2,5154,023,022,SUN-SET TRACT,D,13,14VN81535,14016-20000-13092,Issued,08/13/2014,Bldg-Alter/Repair,Apartment,Plan Check,None,None,VAN NUYS,2014-08-13,415,None,415,None,S,BURLINGTON,AVE,None,1-30,1-30,90057.0,PHOTOVOLTAIC SOLAR PANELS ON ROOF OF (E) APT BLDG,37000.0,None,None,None,None,PERMACITY CONSTRUCTION CORP,5570 W WASHINGTON BLVD,LOS ANGELES,CA,B,827864,JONATHAN,SAUL,PORT,2015-11-30,LINDA,MARTON,None,710 WILSHIRE BLVD,None,"SANTA MONICA, CA",R4-1,None,None,2089.04,1,"(34.06012, -118.26997)",Agent for Owner,5.0,NaN
3,4404,030,010,TR 12086,None,2,None,16044-30000-09658,Permit Finaled,08/29/2016,HVAC,1 or 2 Family Dwelling,No Plan Check,None,None,WEST LA,2016-08-22,315,None,315,None,S,OCEANO,DR,None,None,None,90049.0,None,NaN,None,None,None,None,E/C HEATING AND AIR CONDITION,26888 CUATRO MILPAS ST,VALENCIA,CA,C20,651051,EDY,RUDOLFO,CORDON,2018-07-31,None,None,None,None,None,None,RS-1,None,0,2640.00,11,"(34.05707, -118.4732)",Contractor,NaN,NaN
4,2646,019,011,TR 7158,None,11,None,17042-90000-31792,Permit Finaled,12/28/2017,Plumbing,1 or 2 Family Dwelling,No Plan Check,None,None,INTERNET,2017-12-26,13640,None,13640,None,W,PIERCE,ST,None,None,None,91331.0,None,NaN,None,None,None,None,TITANIUM POWER INC,1545 S LA CIENEGA BLVD,LOS ANGELES,CA,B,989217,DENNIS,HARUO,MIYAHIRA,2017-12-31,YONI,GHERMEZI,None,1545 S LA CIENEGA BLVD,None,"LOS ANGELES, CA",R1-1-O,None,0,1044.03,7,"(34.25487, -118.43002)",Net Applicant,NaN,NaN


In [254]:
data.shape

(9999, 59)

In [93]:
data.shape

(10000, 59)

In [106]:
#data[(data['PCIS Permit #'].duplicated(keep=False)==1)].sort_values(by='PCIS Permit #')

## Clean Data

In [66]:
#data = format_names(data)

# Convert Valuation column to float values
data['Valuation'].fillna(0, inplace=True)
data['Valuation'] = data['Valuation'].astype(float);

In [67]:
data.head()

,Assessor Book,Assessor Page,Assessor Parcel,Tract,Block,Lot,Reference # (Old Permit #),PCIS Permit #,Status,Status Date,Permit Type,Permit Sub-Type,Permit Category,Project Number,Event Code,Initiating Office,Issue Date,Address Start,Address Fraction Start,Address End,Address Fraction End,Street Direction,Street Name,Street Suffix,Suffix Direction,Unit Range Start,Unit Range End,Zip Code,Work Description,Valuation,Floor Area-L.A. Zoning Code Definition,# of Residential Dwelling Units,# of Accessory Dwelling Units,# of Stories,Contractor's Business Name,Contractor Address,Contractor City,Contractor State,License Type,License #,Principal First Name,Principal Middle Name,Principal Last Name,License Expiration Date,Applicant First Name,Applicant Last Name,Applicant Business Name,Applicant Address 1,Applicant Address 2,Applicant Address 3,Zone,Occupancy,Floor Area-L.A. Building Code Definition,Census Tract,Council District,Latitude/Longitude,Applicant Relationship,Existing Code,Proposed Code
0,5408,011,907,TR 10418,None,LT 2,None,15041-10000-21893,Permit Finaled,09/20/2016,Electrical,Commercial,Plan Check,None,None,METRO,11/06/2015,650,None,650,None,N,MAIN,ST,None,None,None,90012,None,0.0,None,None,None,None,M A ELECTRIC,P O BOX 1783,COVINA,CA,C10,569475,MAURICIO,ALONSO,AVELAR,05/31/2017,ABEL,MARIN,ECCO ENGINEERING FIRM,726 W BROADWAY,A,GLENDALE CA,C4-1VL,None,0,2071.00,14,None,Engineer,None,None
1,5151,001,024,TR 21949,None,LT 1,None,17041-90000-32378,Issued,09/11/2017,Electrical,Commercial,No Plan Check,None,None,INTERNET,09/11/2017,261,None,261,None,S,FIGUEROA,ST,None,260,None,90012,None,0.0,None,None,None,None,CANDUIT ELECTRIC INC,8218 HILLROSE ST,SUNLAND,CA,C10,1013159,CHRISTOPHER,MICHAEL,TRUELOVE,09/30/2018,CHRIS,TRUELOVE,None,8218 HILLROSE,None,"SUNLAND, CA",C4-4D,None,0,2075.01,14,"(34.05546, -118.25446)",Net Applicant,None,None
2,5151,014,031,TR 21409,None,LT 1,None,16042-10000-28033,Permit Finaled,02/15/2017,Plumbing,Commercial,No Plan Check,None,None,METRO,12/28/2016,333,None,333,None,S,HOPE,ST,None,23RD FL,None,90071,None,0.0,None,None,None,None,MUIR-CHASE PLUMBING CO INC,4530 BRAZIL STREET,LOS ANGELES,CA,C36,539835,GRANT,DRAKE,MUIR,08/31/2018,None,None,None,None,None,None,C4-4D,None,0,2075.02,14,"(34.05326, -118.25292)",Contractor,None,None
3,5144,021,031,H. F. SPENCER SUBDIVISION OF NORTH 1/2 BLOCK 5...,None,LT NO 3,None,19041-10000-02555,Issued,01/22/2019,Electrical,Commercial,No Plan Check,None,None,METRO,01/22/2019,813,None,813,None,S,FLOWER,ST,None,None,None,90017,None,0.0,None,None,None,None,VELRIO CONSTRUCTION INC,31805 TEMECULA PARKWAY STE 132,TEMECULA,CA,C-7,1017086,FRANCISCO,None,DE JESUS VELEZ,08/31/2020,FRANCISCO,VELEZ,VELRIO CONSTRUCTION INC,None,None,None,C2-4D,None,0,2077.10,14,"(34.04705, -118.26051)",Contractor,None,None
4,5163,018,006,MILLS AND WICKS EXTENSION OF SECOND ST. AND AD...,None,233,18LA03217,16016-10004-17710,Permit Finaled,06/13/2018,Bldg-Alter/Repair,Commercial,Plan Check,None,None,METRO,05/17/2018,1019,None,1019,None,E,4TH,PL,None,None,None,90013,SUPPLEMENTAL TO 16016-10000-17710 WINDOW WASHI...,0.0,None,None,None,None,SHANGRI - LA CONSTRUCTION L P,550 S HOPE ST STE 700,LOS ANGELES,CA,B,926436,BENJAMIN,SALEM,WAHAB,12/31/2018,SHANGRI-LA,CONSTR.,None,550 S. HOPE ST.,700,"LOS ANGELES, CA",M3-1-RIO,None,None,2060.31,14,None,Agent for Contractor,13,None


In [69]:
#data['Valuation'].sort_values().value_counts(sort=False).sort_index()
#data.groupby(['Permit_Type', 'Contractors_Business_Name'])['Valuation'].sum().sort_values(ascending=False).sort_index()

